<a href="https://colab.research.google.com/github/cbaldassari/manifold/blob/main/manifold-informed-wasserstein-barycenter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%%capture
!pip install scikit-learn==1.1.2
!pip install scipy==1.9.1
!pip install missingpy==0.2.0
!pip install yfinance

In [ ]:
import os
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base

import pandas as pd
import numpy as np
import statsmodels.api as sm
import datetime
from sklearn import mixture
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
from scipy.stats import skew
from scipy.stats import kurtosis
from tqdm import tqdm
from missingpy import MissForest
import yfinance as yf

In [ ]:
yahoo = yf.download('SPY', start='2018-01-01', end='2022-12-31')

[*********************100%%**********************]  1 of 1 completed


In [ ]:
df1=pd.read_excel("/data/ice_electric-2018final.xlsx")
df2=pd.read_excel("/data/ice_electric-2019final.xlsx")
df3=pd.read_excel("/data/ice_electric-2020final.xlsx")
df4=pd.read_excel("/data/ice_electric-2021final.xlsx")
df5=pd.read_excel("/data/ice_electric-2022final.xlsx")

In [ ]:
df = pd.concat([df1,df2,df3,df4,df5], ignore_index=True, sort=False)
elec=df[["Price hub","Trade date","Wtd avg price $/MWh"]]

In [ ]:
dfe1=elec[elec['Price hub']=="PJM WH Real Time Peak"]
dfe1["hub"]="pjm"

dfe2=elec[elec['Price hub']=="Palo Verde Peak"]
dfe2["hub"]="paloverde"

dfe3=elec[elec['Price hub']=="SP15 EZ Gen DA LMP Peak"]
dfe3["hub"]="sp15"

dfe4=elec[elec['Price hub']=="Nepool MH DA LMP Peak"]
dfe4["hub"]="nepool"

dfe5=elec[elec['Price hub']=="Mid C Peak"]
dfe5["hub"]="midcolumbia"

#dfe6=elec[elec['Price hub']=="PJM WH Real Time Peak"]
#dfe6["hub"]="pjm"

dfe = pd.concat([dfe1,dfe2,dfe3,dfe4,dfe5], ignore_index=True, sort=False)

<ipython-input-7-6773057c8296>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe1["hub"]="pjm"
<ipython-input-7-6773057c8296>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfe2["hub"]="paloverde"
<ipython-input-7-6773057c8296>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

##missForest

##Adapt different series columns' name

In [ ]:
nepool=dfe[dfe["hub"]=="nepool"]
nepool.reset_index(inplace=True)
nepool.drop(columns=["Price hub","hub","index"],inplace=True)
nepool.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
nepool

<ipython-input-8-d2443199ca92>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nepool.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-8-d2443199ca92>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nepool.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,199.71
1,2018-01-04 00:00:00,311.14
2,2018-01-05 00:00:00,149.08
3,2018-01-09 00:00:00,89.25
4,2018-01-10 00:00:00,53.50
...,...,...
1106,2022-12-19 00:00:00,135.83
1107,2022-12-20 00:00:00,110.67
1108,2022-12-21 00:00:00,81.68
1109,2022-12-22 00:00:00,209.63


In [ ]:
pjm=dfe[dfe["hub"]=="pjm"]
pjm.reset_index(inplace=True)
pjm.drop(columns=["Price hub","hub","index"],inplace=True)
pjm.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
pjm

<ipython-input-9-83127bb27ac2>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pjm.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-9-83127bb27ac2>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pjm.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,172.81
1,2018-01-04 00:00:00,366.91
2,2018-01-05 00:00:00,79.43
3,2018-01-09 00:00:00,36.31
4,2018-01-10 00:00:00,29.31
...,...,...
1221,2022-12-19 00:00:00,85.45
1222,2022-12-20 00:00:00,79.86
1223,2022-12-21 00:00:00,69.11
1224,2022-12-22 00:00:00,181.75


In [ ]:
sp15=dfe[dfe["hub"]=="sp15"]
sp15.reset_index(inplace=True)
sp15.drop(columns=["Price hub","hub","index"],inplace=True)
sp15.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
sp15

<ipython-input-10-a4e1f6993e0b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp15.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-10-a4e1f6993e0b>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sp15.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,50.96
1,2018-01-04 00:00:00,36.42
2,2018-01-05 00:00:00,37.26
3,2018-01-09 00:00:00,33.00
4,2018-01-10 00:00:00,36.70
...,...,...
1172,2022-12-19 00:00:00,320.10
1173,2022-12-20 00:00:00,277.38
1174,2022-12-21 00:00:00,365.77
1175,2022-12-22 00:00:00,190.00


In [ ]:
paloverde=dfe[dfe["hub"]=="paloverde"]
paloverde.reset_index(inplace=True)
paloverde.drop(columns=["Price hub","hub","index"],inplace=True)
paloverde.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)
paloverde

<ipython-input-11-eaec451fa2be>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paloverde.drop(columns=["Price hub","hub","index"],inplace=True)
<ipython-input-11-eaec451fa2be>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  paloverde.rename(columns={"Wtd avg price $/MWh": "Price", "Trade date":"Date"}, inplace=True)


,Date,Price
0,2018-01-03 00:00:00,42.78
1,2018-01-04 00:00:00,29.80
2,2018-01-05 00:00:00,26.28
3,2018-01-09 00:00:00,22.01
4,2018-01-10 00:00:00,23.69
...,...,...
1225,2022-12-19 00:00:00,323.14
1226,2022-12-20 00:00:00,275.27
1227,2022-12-21 00:00:00,320.22
1228,2022-12-22 00:00:00,181.83


In [ ]:
spx=pd.DataFrame(yahoo["Close"])
spx.reset_index(inplace=True)
spx.rename(columns={"Close":'Price'}, inplace=True)
spx

,Date,Price
0,2018-01-02,268.769989
1,2018-01-03,270.470001
2,2018-01-04,271.609985
3,2018-01-05,273.420013
4,2018-01-08,273.920013
...,...,...
1254,2022-12-23,382.910004
1255,2022-12-27,381.399994
1256,2022-12-28,376.660004
1257,2022-12-29,383.440002


In [ ]:
oil=pd.read_excel("/data/oil.xls",sheet_name="Data 1", parse_dates=True)

In [ ]:
gas=pd.read_excel("/data/gas.xls",sheet_name="Data 1", parse_dates=True)

In [ ]:
terms=pd.read_excel("/data/LW_daily.xlsx", parse_dates=["Date"])

In [ ]:
multi_ts_dic={
    "sp15":sp15,
    "nepool":nepool,
    "paloverde":paloverde,
    "pjm":pjm,
    "spx":spx,
    "gas": gas,
    "oil": oil

    }

'\n"term1m":term1m,\n"term6m":term6m,\n"term1Yr":term1Yr,\n"term5Yr":term5Yr,\n"term7Yr":term7Yr,\n"term10Yr":term10Yr,\n"term20Yr":term20Yr\n'

In [ ]:
stocha=[]
hubfilled=pd.DataFrame()
stochastic2=pd.DataFrame()
stochastic3=pd.DataFrame()
stochastic4=pd.DataFrame()
logprezzi=pd.DataFrame()
trend=pd.DataFrame()
prezzi=pd.DataFrame()
flagin=True
chart=pd.DataFrame()
logprezzidepurati=pd.DataFrame()
logprezzi=pd.DataFrame()
filLogPrices=pd.DataFrame()
stochastic=pd.DataFrame()
x=pd.DataFrame()
r=pd.DataFrame()
linechart=pd.DataFrame()
prezzifillati=pd.DataFrame()

In [ ]:
%%capture
for hub in multi_ts_dic:
  print(hub)
  huball=multi_ts_dic[hub]
  #huball.drop(columns=['Price hub'], inplace=True)
  huball.rename(columns={'Wtd avg price $/MWh':'Price',  'Trade date':'Date'}, inplace=True)
  print(huball.Price[huball.Price<=0])
  huball.Price[huball.Price<=0]=np.nan
  print(huball.Price[huball.Price<=0])
  lowess=pd.DataFrame()

  #huballhub=huball[huball['hub_transcode'] == hub]      #ritaglio il singolo hub -> missForest cambierebbe
  huball=huball.groupby('Date').first() #group by per record doppi -> prendo il primo

  #Creo la griglia e sovrappongo la serie dei prezzi
  start = '01-01-2018'
  end = '31-12-2022'
  grid=pd.Series(pd.date_range(start = start, end = end, freq='D'))
  grid.sort_index(ascending=True)
  huball= huball.reindex(grid, fill_value=np.nan)
  huball.index.name = "Date"

  linechart[hub]=huball["Price"]

  toimpute=pd.DataFrame()
  toimpute["Date"]=np.arange(len(huball))
  toimpute["Price"]=huball["Price"].values

  imputer = MissForest(random_state=1234)
  X_imputed = imputer.fit_transform(toimpute.values)

  filLogPrices[hub]=np.log(X_imputed[:,1])
  prezzifillati[hub]=X_imputed[:,1]
  trend[hub] = sm.nonparametric.lowess(filLogPrices[hub], filLogPrices.index, frac=0.1, it=10, is_sorted=True, return_sorted=False, missing="raise")
  stochastic[hub] = filLogPrices[hub]-trend[hub].values

  logprezzidepurati[hub]=stochastic[hub]
  logprezzi[hub]=filLogPrices[hub]

  x[hub]=stochastic[hub]
  r[hub]=stochastic[hub]-stochastic[hub].shift(1)
  print("xxxxxx",x.shape)
  print("rrrrrr",r.shape)

filLogPrices["Date"]=huball.index
filLogPrices.set_index("Date", inplace=True)

logprezzidepurati["Date"]=huball.index
logprezzidepurati.set_index("Date", inplace=True)

logprezzi["Date"]=huball.index
logprezzi.set_index("Date", inplace=True)

x["Date"]=huball.index
x.set_index("Date", inplace=True)

r["Date"]=huball.index
r.set_index("Date", inplace=True)
r.dropna(inplace=True)

In [ ]:
terms=pd.read_excel("/data/LW_daily.xlsx", parse_dates=["Date"])

 ##Terms

In [ ]:
terms

,Date,1 m,2 m,3 m,4 m,5 m,6 m,7 m,8 m,9 m,...,351 m,352 m,353 m,354 m,355 m,356 m,357 m,358 m,359 m,360 m
0,1961-06-14,2.245463,2.312255,2.381113,2.451428,2.522306,2.592520,2.660420,2.723988,2.781463,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1961-06-15,2.238297,2.305464,2.373091,2.441878,2.511999,2.582966,2.653560,2.721918,2.785903,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1961-06-16,2.235452,2.303258,2.371947,2.442190,2.513979,2.586526,2.658266,2.727051,2.790648,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1961-06-19,2.217626,2.295898,2.371827,2.446512,2.520499,2.593621,2.664923,2.732801,2.795486,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1961-06-20,2.182799,2.270473,2.353695,2.433896,2.511963,2.588052,2.661447,2.730642,2.793852,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15362,2022-12-23,3.867337,4.023574,4.154775,4.259353,4.338186,4.394684,4.433878,4.460945,4.480016,...,3.816482,3.813802,3.811096,3.808366,3.805612,3.802834,3.800036,3.797223,3.794403,3.791584
15363,2022-12-27,3.903516,4.073273,4.213900,4.324662,4.407392,4.466296,4.506835,4.534270,4.552698,...,3.919447,3.916438,3.913439,3.910452,3.907474,3.904505,3.901540,3.898580,3.895626,3.892682
15364,2022-12-28,3.995884,4.136985,4.255885,4.351028,4.423119,4.474977,4.510701,4.534563,4.550189,...,3.953790,3.950572,3.947351,3.944128,3.940907,3.937687,3.934469,3.931256,3.928051,3.924859
15365,2022-12-29,4.060537,4.183430,4.287693,4.372657,4.438962,4.488623,4.524618,4.550142,4.567915,...,3.876828,3.873405,3.869989,3.866585,3.863192,3.859810,3.856438,3.853076,3.849724,3.846386


#missForest

In [ ]:
minrangedate = '01-01-2018'
maxrangedate = '31-12-2022'

In [ ]:
deb=terms.copy()

In [ ]:
deb.set_index("Date",inplace=True)

In [ ]:
gridate=pd.Series(pd.date_range(start = (minrangedate), end = maxrangedate, freq='D'))
deb = deb.reindex(gridate, fill_value=np.nan)
#deb.index.names = ['Date']

<ipython-input-86-b20f555e693f>:1: UserWarning: Parsing dates in DD/MM/YYYY format when dayfirst=False (the default) was specified. This may lead to inconsistently parsed dates! Specify a format to ensure consistent parsing.
  gridate=pd.Series(pd.date_range(start = (minrangedate), end = maxrangedate, freq='D'))


In [ ]:
deb

,1 m,2 m,3 m,4 m,5 m,6 m,7 m,8 m,9 m,10 m,...,351 m,352 m,353 m,354 m,355 m,356 m,357 m,358 m,359 m,360 m
2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2018-01-02,1.261483,1.316193,1.372369,1.429035,1.485087,1.539317,1.590661,1.638366,1.681890,1.720710,...,2.887749,2.889039,2.890414,2.891862,2.893373,2.894938,2.896548,2.898196,2.899876,2.901583
2018-01-03,1.253046,1.312112,1.370645,1.428207,1.484243,1.538031,1.588865,1.636204,1.679638,1.718733,...,2.856631,2.857915,2.859285,2.860730,2.862238,2.863798,2.865402,2.867043,2.868713,2.870408
2018-01-04,1.265548,1.318945,1.374632,1.431363,1.487743,1.542289,1.593775,1.641460,1.684964,1.724010,...,2.858304,2.859605,2.861001,2.862478,2.864023,2.865626,2.867278,2.868971,2.870698,2.872453
2018-01-05,1.259828,1.316978,1.375046,1.432954,1.489574,1.543774,1.594696,1.641876,1.685093,1.724135,...,2.882236,2.883480,2.884810,2.886217,2.887693,2.889233,2.890831,2.892479,2.894173,2.895906
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-27,3.903516,4.073273,4.213900,4.324662,4.407392,4.466296,4.506835,4.534270,4.552698,4.564882,...,3.919447,3.916438,3.913439,3.910452,3.907474,3.904505,3.901540,3.898580,3.895626,3.892682
2022-12-28,3.995884,4.136985,4.255885,4.351028,4.423119,4.474977,4.510701,4.534563,4.550189,4.560310,...,3.953790,3.950572,3.947351,3.944128,3.940907,3.937687,3.934469,3.931256,3.928051,3.924859
2022-12-29,4.060537,4.183430,4.287693,4.372657,4.438962,4.488623,4.524618,4.550142,4.567915,4.579877,...,3.876828,3.873405,3.869989,3.866585,3.863192,3.859810,3.856438,3.853076,3.849724,3.846386
2022-12-30,3.975668,4.128433,4.256138,4.358919,4.438403,4.497778,4.541233,4.572964,4.596294,4.613331,...,3.935679,3.932788,3.929963,3.927201,3.924494,3.921827,3.919184,3.916551,3.913912,3.911259


In [ ]:
hubnames=['  6 m',"120 m"]
log=pd.DataFrame()
for hub in hubnames:
  imputer = MissForest(random_state=1234)
  singlegrid=deb[hub].values
  idxs=np.arange(singlegrid.shape[0])
  toimpute=pd.DataFrame()

  toimpute["Date"]=idxs
  toimpute["Price"]=singlegrid
  X_imputed = imputer.fit_transform(toimpute.values)

  filled=pd.DataFrame(X_imputed[:,1])
  filled.set_index(deb[hub].index, inplace=True)
  log[hub]=filled.reindex()

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 0


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


...

Iteration: 0


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 1


/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:400: FutureWarning: Criterion 'mse' was deprecated in v1.0 and will be removed in version 1.2. Use `criterion='squared_error'` which is equivalent.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Iteration: 2


In [ ]:
reterm=log.diff()

In [ ]:
reterm=reterm.reindex()

In [ ]:
reterm.dropna(inplace=True)

In [ ]:
reterm

,6 m,10 m
2018-01-02,-0.000241,-0.000027
2018-01-03,-0.001286,-0.001977
2018-01-04,0.004258,0.005276
2018-01-05,0.001485,0.000126
2018-01-06,0.000338,0.000125
...,...,...
2022-12-27,0.006365,0.010619
2022-12-28,0.008681,-0.004572
2022-12-29,0.013646,0.019567
2022-12-30,0.009155,0.033453


In [ ]:
r.reset_index(drop=True)
reterm.reset_index(drop=True)

,6 m,10 m
0,-0.000241,-0.000027
1,-0.001286,-0.001977
2,0.004258,0.005276
3,0.001485,0.000126
4,0.000338,0.000125
...,...,...
1820,0.006365,0.010619
1821,0.008681,-0.004572
1822,0.013646,0.019567
1823,0.009155,0.033453


In [ ]:
dataset=pd.concat([r,reterm],axis=1)

In [ ]:
dataset.to_excel("/data/chartData/paper6_dataset.xlsx")